# DiCE XAI

In [ ]:
import dice_ml
import numpy as np
import pandas as pd

from dice_ml import Dice
from dice_ml.utils import helpers  

In [ ]:
def load_dataDICE(filepath1, filepath2, top_k):
    
    stand = StandardScaler()
    selector = SelectKBest(f_classif, k=top_k)
    
    data1 = pd.read_pickle(filepath1)
    data_df1 = pd.DataFrame(data1)
    features_df1 = data_df1[data_df1.columns[1:]]
    labels1 = data_df1[data_df1.columns[0]]
    
    data2 = pd.read_pickle(filepath2)
    data_df2 = pd.DataFrame(data2)
    features_df2 = data_df2[data_df2.columns[1:]]
    labels2 = data_df2[data_df2.columns[0]]

    ### Train - 2023 S1 ### 
    X_train_std = stand.fit_transform(features_df1) 
    X_train_selected = selector.fit_transform(X_train_std, labels1)
    
    cols_idxs = selector.get_support(indices=True)
    features_selected = features_df1.iloc[:,cols_idxs]
    features_selectedName = features_selected.columns.tolist()
    
    X_train_selected = pd.DataFrame(X_train_selected, columns=features_selectedName)
    X_train_stdFull = pd.concat([X_train_selected.reset_index(drop=True), labels1.reset_index(drop=True)], axis=1)
         
    ### Test - 2023 S2 ### 
    X_test_std = stand.transform(features_df2)
    X_test_selected = selector.transform(X_test_std)
    X_test_selected = pd.DataFrame(X_test_selected, columns=features_selectedName)
    
    return X_train_selected, X_test_selected, features_selectedName, X_train_stdFull

In [ ]:
# data_file1, data_file2: The data files used in this experiment.
# trainedModel: The trained model used in this study(XGBoost Classifier).
# topk: Top-K number.
# usedSample: Randomly selected Pandas Dataframe indices. 
#             In this study, we will randomly group (random sampling) students classified as at-risk by the XGBoost Classifier 
#             and input each group into DiCE to generate counterfactual instances corresponding to each case. 
def setDiCE(data_file1, data_file2, trainedModel, topk, usedSample):
    
    X_train_selected, X_test_selected, features_selectedName, X_train_stdFull = load_dataDICE(data_file1, data_file2, topk)
    
    # Extract the corresponding rows based on the index.
    X_test_selected = X_test_selected.iloc[usedSample]

    # features_list1: Set continuous features.
    # features_list2: Set features that need modification. If the feature set includes some unchangeable features (such as gender, age), this parameter can be used for setting.
    # Set up DiCE according to the official DiCE documentation
    xai_dataset = dice_ml.Data(dataframe=X_train_stdFull, continuous_features=features_list1, features_to_vary=features_list2, outcome_name='label')
    xai_model = dice_ml.Model(model=trainedModel, backend="sklearn")
    exp_explanation = dice_ml.Dice(xai_dataset, xai_model, method="random")
    
    # valueRange: Set upper and lower numerical limits for each feature to prevent generating counterfactual instances that exceed the numerical range.
    # seed： Set seed
    dice_explanation = exp_explanation.generate_counterfactuals(X_test_selected, total_CFs=10, desired_class=1, verbose=False, permitted_range=valueRange, random_seed=seed) 
    dice_explanationJSON = dice_explanation.to_json()
    
    # After DiCE returns results in JSON format, extract the counterfactual instance, reverse the Z-Score number, and construct the request as part of the prompt that will be input into ChatGPT.
    return dice_explanationJSON

# ChatGPT

In [ ]:
import pandas as pd
import pickle

from openai import OpenAI

client = OpenAI()

# request： 
def generate_feedback(request):
    
    # GPT - 4 Models
    modelName = "" 
    
    # The prompts inputted into the model, including the roles that need to be played as well as the rules that need to be followed for writing feedback.
    conversation_list = [
        {"role": "system", "content": "Please act as a teacher to provide feedback for students. The feedback must adhere to the following rules:"},
        {"role": "system", "content": "Feedback should strengthen the relationships between teachers and learners, and aim to build social relationships with learners by using a polite, positive tone."},
        {"role": "system", "content": "Feedback should highlight the strengths of learners’ performance, acknowledging their capabilities to enhance self-efficacy."},
        {"role": "system", "content": "Feedback should provide critiques regarding learners' insufficient performance while avoiding the use of harsh wording."},
        {"role": "system", "content": "Feedback should provide actionable suggestions regarding learners' insufficient performance, helping them achieve their expected learning goals."},
        {"role": "system", "content": "Feedback should encourage learner agency."},
        {"role": "system", "content": "Feedback should encourage positive learner affect, showing an emotional response and care on learners’ feelings."},
        {"role": "system", "content": "Feedback should promote learner independence by rephrasing comments as questions or suggestions rather than statements, encouraging learners to think for themselves."},
        {"role": "system", "content": "Feedback should encourage learners to ask questions, seeking help from others to engage in constructive feedback dialogues."},
        {"role": "system", "content": "Feedback should be accessible and comprehensible for learners."},
        {"role": "system", "content": "Feedback must not exceed 225 words."},
        {"role": "system", "content": "Feedback must provided in the form of an email."},
        {"role": "user", "content": request}]
    
        ### Request Example:
        # The feedback is based on the students’ higher-than-median learning activities: 1.good performance: Good Statement 1, Good Statement 2.
        # , and the under-performed learning activities with the corre- sponding suggestions: Insufficient Statement 1, suggest changing it to Suggestion 1. Insufficient Statement 2, suggest changing it to Suggestion 2.
        ### 
        
    # Return generated feedback
    response = client.chat.completions.create(model=modelName, messages=conversation_list)
    feedback = response.choices[0].message.content
    return feedback